<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/geneformer_training_with_dop_neuron_pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scanpy anndata umap-learn python-igraph louvain h5py leidenalg > /dev/null 2>&1

In [2]:
from google.colab import drive
import os
import scanpy as sc
import pandas as pd
from scipy.io import mmread
import scipy.sparse
import pickle

In [21]:
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/HODD/")

with open("dopamine_subset.pkl", "rb") as f:
    adata = pickle.load(f)

adata

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AnnData object with n_obs × n_vars = 21624 × 41625
    obs: 'libname', 'biosample_id', 'donor_id', 'species', 'species__ontology_label', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'sex', 'date', 'Donor_Age', 'Donor_PMI', 'Status', 'Cause_of_Death', 'FACS_Classification', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'leiden', 'merged_clusters'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', '

In [22]:
# sc.pp.normalize_total(adata, target_sum=1e4)  # Normalize to a total count per cell
# sc.pp.log1p(adata)  # Log-transform the data

In [23]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)  # You can choose how many genes to keep
adata = adata[:, adata.var['highly_variable']]  # Keep only HVGs

In [24]:
sc.pp.scale(adata, max_value=10)  # Optional: scaling genes to zero mean and unit variance

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_scale.py:317: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [19]:
# Filter the AnnData object for only "normal" and "Parkinson disease"
adata = adata[adata.obs["disease__ontology_label"].isin(["normal", "Parkinson disease"])]

adata.obs['disease__ontology_label'].value_counts()

,count
disease__ontology_label,
normal,15774
Parkinson disease,2263


In [25]:
adata.obs.rename(columns={"disease__ontology_label": "disease_state"}, inplace=True)

In [8]:
!git lfs install

Git LFS initialized.


In [9]:
!git clone https://huggingface.co/ctheodoris/Geneformer

fatal: destination path 'Geneformer' already exists and is not an empty directory.


In [10]:
%cd Geneformer

/content/drive/MyDrive/HODD/Geneformer


In [11]:
!pip install .

Processing /content/drive/MyDrive/HODD/Geneformer
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.

In [1]:
from geneformer import TranscriptomeTokenizer

# Prepare the data using Geneformer Preprocessor

data_directory = '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files'
output_path = data_directory
ouput_prefix = 'dopamine_subset_tokenized'

tk = TranscriptomeTokenizer({"disease__ontology_label": "disease_state"}, nproc=4)
tk.tokenize_data(data_directory, output_path, ouput_prefix, file_format="h5ad")

Tokenizing /content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad


AssertionError: 'ensembl_id' column missing from data.var

In [16]:
from geneformer import TranscriptomeTokenizer

In [ ]:
from geneformer import Geneformer

# Load pretrained Geneformer model
model = Geneformer.from_pretrained("geneformer")

# Fine-tune on your dataset
model.train(tokenized_data, output_dir="fine_tuned_geneformer")

In [ ]:
perturbed_data = preprocessor.apply_perturbation(adata.X, genes_to_perturb=["GeneA", "GeneB"])
perturbed_predictions = model.predict(perturbed_data)
